In [4]:
import sys
sys.path.append('/Users/luizfelipedossantos/code/micheledeantoni/APP/')

import requests
from bs4 import BeautifulSoup
import pandas as pd

from player_profiling.data import load_data_fifa23

In [88]:
df = load_data_fifa23()

In [100]:
df.head(24)

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/230009,23,9,2023-01-13,L. Messi,Lionel Andrés Messi Cuccittini,RW,91,91,...,63+3,63+3,64+3,59+3,50+3,50+3,50+3,59+3,19+3,https://cdn.sofifa.net/players/158/023/23_120.png
1,165153,/player/165153/karim-benzema/230009,23,9,2023-01-13,K. Benzema,Karim Benzema,"CF, ST",91,91,...,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3,https://cdn.sofifa.net/players/165/153/23_120.png
2,188545,/player/188545/robert-lewandowski/230009,23,9,2023-01-13,R. Lewandowski,Robert Lewandowski,ST,91,91,...,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/23_120.png
3,192985,/player/192985/kevin-de-bruyne/230009,23,9,2023-01-13,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,...,79+3,79+3,78+3,74+3,68+3,68+3,68+3,74+3,21+3,https://cdn.sofifa.net/players/192/985/23_120.png
4,231747,/player/231747/kylian-mbappe/230009,23,9,2023-01-13,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,95,...,63+3,63+3,67+3,63+3,54+3,54+3,54+3,63+3,18+3,https://cdn.sofifa.net/players/231/747/23_120.png
5,192119,/player/192119/thibaut-courtois/230009,23,9,2023-01-13,T. Courtois,Thibaut Nicolas Marc Courtois,GK,90,91,...,31+3,31+3,29+3,29+3,29+3,29+3,29+3,29+3,87+3,https://cdn.sofifa.net/players/192/119/23_120.png
6,209331,/player/209331/mohamed-salah/230009,23,9,2023-01-13,M. Salah,Mohamed Salah Ghaly,RW,90,90,...,68+3,68+3,71+3,67+3,58+3,58+3,58+3,67+3,22+3,https://cdn.sofifa.net/players/209/331/23_120.png
7,167495,/player/167495/manuel-neuer/230009,23,9,2023-01-13,M. Neuer,Manuel Peter Neuer,GK,89,89,...,43+3,43+3,36+3,34+3,34+3,34+3,34+3,34+3,87+2,https://cdn.sofifa.net/players/167/495/23_120.png
8,190871,/player/190871/neymar-da-silva-santos-jr/230009,23,9,2023-01-13,Neymar Jr,Neymar da Silva Santos Júnior,LW,89,89,...,63+3,63+3,66+3,61+3,50+3,50+3,50+3,61+3,20+3,https://cdn.sofifa.net/players/190/871/23_120.png
9,200145,/player/200145/carlos-henrique-venancio-casimi...,23,9,2023-01-13,Casemiro,Carlos Henrique Venancio Casimiro,CDM,89,89,...,86+3,86+3,81+3,82+3,87+2,87+2,87+2,82+3,21+3,https://cdn.sofifa.net/players/200/145/23_120.png


In [91]:
player = df.iloc[23]
player

player_id                                                      200104
player_url                        /player/200104/heung-min-son/230009
fifa_version                                                       23
fifa_update                                                         9
fifa_update_date                                           2023-01-13
                                          ...                        
cb                                                               54+3
rcb                                                              54+3
rb                                                               63+3
gk                                                               19+3
player_face_url     https://cdn.sofifa.net/players/200/104/23_120.png
Name: 23, Length: 110, dtype: object

In [56]:
base_url = "https://www.transfermarkt.de"

In [50]:
base_search_url = "https://www.transfermarkt.de/schnellsuche/ergebnis/schnellsuche?query="

In [51]:
player_name = player['player_url'].split("/")[3]
player_name = player_name.replace("-", "+")

In [52]:
search_url = base_search_url + player_name
search_url

'https://www.transfermarkt.de/schnellsuche/ergebnis/schnellsuche?query=lionel+messi'

In [102]:
def get_player_data(search_url):
    try:
        headers = {'User-Agent':
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

        player_data = []

        response = requests.get(search_url, headers=headers)

        soup = BeautifulSoup(response.content, "html.parser")

        player_url = soup.find("td", class_="hauptlink").find("a")['href']
        player_url = player_url.replace("profil", "leistungsdatendetails")
        player_url = player_url + '/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'

        details_player_url = base_url + player_url

        response = requests.get(details_player_url, headers=headers)

        player_season_data = []

        soup = BeautifulSoup(response.content, "html.parser")

        for row in soup.find("table", class_="items").find("tbody").find_all("tr", class_="odd"):
            for column in row.find_all("td", class_="zentriert"):
                player_season_data.append(column.string)

            season = player_season_data[0]
            pps = float(player_season_data[4].replace("-", "0").replace(",", "."))
            goals = int(player_season_data[5].replace("-", "0"))
            assists = int(player_season_data[6].replace("-", "0"))
            player_data.append({'season': season, 'PPS': pps, 'goals': goals, 'assists': assists})
            player_season_data = []

        for row in soup.find("table", class_="items").find("tbody").find_all("tr", class_="even"):
            for column in row.find_all("td", class_="zentriert"):
                player_season_data.append(column.string)

            season = player_season_data[0]
            pps = float(player_season_data[4].replace("-", "0").replace(",", "."))
            goals = int(player_season_data[5].replace("-", "0"))
            assists = int(player_season_data[6].replace("-", "0"))
            player_data.append({'season': season, 'PPS': pps, 'goals': goals, 'assists': assists})
            player_season_data = []
    except:
        return player_data

    return player_data


In [107]:
#for index, row in df.iterrows():
#    print(row['c1'], row['c2'])

player = df.iloc[23]
player_name = player['player_url'].split("/")[3]
player_name = player_name.replace("-", "+")
search_url = base_search_url + player_name

player_data = get_player_data(search_url)

# Search for short name
if len(player_data) == 0:
    player_name = player['short_name'].lower().replace(" ", "+")
    search_url = base_search_url + player_name
    player_data = get_player_data(search_url)

    # Cases like Heung-min Son
    if len(player_data) == 0:
        player_name = player['player_url'].split("/")[3]
        player_name = '+'.join(player_name.rsplit('-', 1))
        search_url = base_search_url + player_name
        player_data = get_player_data(search_url)

print(player_data)

https://www.transfermarkt.de/schnellsuche/ergebnis/schnellsuche?query=heung+min+son
['heung+min+son']
https://www.transfermarkt.de/schnellsuche/ergebnis/schnellsuche?query=heung+min+son
https://www.transfermarkt.de/schnellsuche/ergebnis/schnellsuche?query=heung-min+son
[{'season': '23/24', 'PPS': 1.8, 'goals': 9, 'assists': 2}, {'season': '22/23', 'PPS': 1.58, 'goals': 10, 'assists': 6}, {'season': '22/23', 'PPS': 2.0, 'goals': 2, 'assists': 0}, {'season': '21/22', 'PPS': 1.5, 'goals': 0, 'assists': 0}, {'season': '21/22', 'PPS': 2.0, 'goals': 1, 'assists': 1}, {'season': '20/21', 'PPS': 1.59, 'goals': 17, 'assists': 10}, {'season': '20/21', 'PPS': 2.29, 'goals': 3, 'assists': 1}, {'season': '20/21', 'PPS': 3.0, 'goals': 1, 'assists': 2}, {'season': '19/20', 'PPS': 2.0, 'goals': 2, 'assists': 0}, {'season': '19/20', 'PPS': 0.0, 'goals': 0, 'assists': 0}, {'season': '18/19', 'PPS': 1.87, 'goals': 12, 'assists': 7}, {'season': '18/19', 'PPS': 3.0, 'goals': 1, 'assists': 2}, {'season': '1